In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0


In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units',min_value=32,max_value=512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))
    
    #  Tune the learning rate for the optimizer
    #  Choose an optimal value from 0.01,0.001,0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                 loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [6]:
tuner = kt.Hyperband(model_builder, objective = 'val_accuracy',max_epochs = 10, factor = 3, directory = 'my_dir', project_name = 'intro_to_kt')

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 16s]
val_accuracy: 0.8628000020980835

Best val_accuracy So Far: 0.8828999996185303
Total elapsed time: 00h 04m 04s
INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 288 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4844 - accuracy: 0.8295 - val_loss: 0.4175 - val_accuracy: 0.8479
Epoch 2/10
1875/1875 [==============================] - 2s 975us/step - loss: 0.3635 - accuracy: 0.8687 - val_loss: 0.3708 - val_accuracy: 0.8688
Epoch 3/10
1875/1875 [==============================] - 2s 997us/step - loss: 0.3263 - accuracy: 0.8790 - val_loss: 0.3757 - val_accuracy: 0.8681
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3023 - accuracy: 0.8880 - val_loss: 0.3598 - val_accuracy: 0.8709
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2850 - accuracy: 0.8947 - val_loss: 0.3474 - val_accuracy: 0.8757
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2671 - accuracy: 0.9016 - val_loss: 0.3372 - val_accuracy: 0.8820
Epoch 7/10
1875/1875 [==============================] - 2s 980us/step - loss: 0.2566 - accuracy: 0.9040 - val_loss: 0.3374 - val_acc